In [1]:
#%%
import pandas as pd
import numpy as np
import xlwings as xw
import os
import re
from pathlib import Path
from datetime import datetime
import requests
import hide
import functions

In [2]:
# Where is the file?
# file = 'J12209 MODEC - EQUINOR BACALHAU FPSO - COM R7.xlsm'
file = 'J12473 MODEC - 4103 URAU FPSO - ENT R1.xlsm'
path = '/Users/infowizard/Downloads/'
file_path = Path(path, file)

In [26]:
# Read and initialize values
# Differentiate between new and legacy template
try:
    wb = xw.Book(file_path, password=hide.legacy)
    template = 'legacy'
except:
    wb = xw.Book(file_path, password=hide.new)
    template = 'new'
visible_sheets = [sht.name for sht in wb.sheets if sht.visible]
full_column_list = ['NO', 'SN', 'Description', 'Qty', 'Unit', 'Unit Price', 'Subtotal Price', 'Scope', 'Model',
                    'Cur', 'UC', 'SC', 'Discount', 'UCD', 'SCD', 'Remark', 'Rate', 'UCDQ', 'SCDQ', 'BUCQ', 'BSCQ',
                    'Default', 'Warranty', 'Freight', 'Special', 'Risk', 'MU', 'FUP', 'RUPQ', 'RSPQ', 'UPLS', 'SPLS', 'Profit', 
                    'Margin', 'Maker', 'Lumpsum', 'Leadtime', 'Format', 'Category', 'System']
df = pd.DataFrame(columns=full_column_list)
risk = 0.05
# Read and set currency from FX sheet
fx = wb.sheets['FX']
exchange_rates = dict(fx.range('A2:B9').value)
quoted_currency = fx.range('B12').value
project_info = dict(fx.range('A36:B45').value)
# Read system sheets
cols = ['NO', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal Price', 'Model', 'Cur', 'UC', 'SC', 'Discount']
systems = pd.DataFrame()
defaults = {}
system_names = []
skip_sheets = ['FX', 'Cover', 'Intro', 'ES', 'T&C']
for sheet in visible_sheets:
    if sheet not in skip_sheets:
        system_names.append(sheet.upper())
        ws = wb.sheets[sheet]
        escalation = dict(ws.range('K2:L5').value)
        default_mu = ws.range('H5').value
        escalation['default_mu'] = default_mu
        defaults[sheet.upper()] = escalation
        last_row = ws.range('D100000').end('up').row  #Returns a number
        last_cell = 'K' + str(last_row) 
        data = ws.range('A8:' + last_cell).options(pd.DataFrame, index=False).value
        data.columns = cols
        data['System'] = str(sheet.upper())
        data['Category'] = 'Product'
        systems = pd.concat([systems, data], join='outer')
systems = pd.concat([systems, df], join='outer')
systems = systems.reindex(columns=full_column_list)
       
# Read Engineering Services
es_cols = ['NO', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal Price', 'Model', 'Cur', 'UC', 'SC', 'Discount']
es = wb.sheets['ES']
es_last_row = es.range('D100000').end('up').row
es_last_cell = 'K' + str(es_last_row)
eng_service = es.range('A8:' + es_last_cell).options(pd.DataFrame, index=False).value
eng_service.columns = es_cols
eng_service = pd.concat([eng_service, df], join='outer')
eng_service = eng_service.reindex(columns=full_column_list)

In [29]:
eng_service['Discount'] = np.nan

In [30]:
xw.view(eng_service)

In [4]:
# Remove lineitem numbers
systems = systems.reset_index(drop=True)
for idx in systems.index:
    if str(systems.loc[idx, 'NO']).count('.') == 2:
        systems.loc[idx, 'NO'] = np.nan

for idx in systems.index:
    if pd.notna(systems.loc[idx, 'NO']) and not pd.notna(systems.loc[idx, 'Qty']):
        systems.loc[idx, 'NO'] = np.nan

In [5]:
# Let's take care of the main numbering
systems['Format'] = np.nan
item_count = 10
for idx in systems.index:
    if pd.notna(systems.loc[idx, 'NO']):
        try:
            systems.at[idx, 'NO'] = item_count
            systems.at[idx, 'Format'] = 'Item'
            item_count += 10
        except Exception as e:
            print(str(e))
            pass


In [6]:
# Let's do Serial Number
# Lineitems must have values in Description, Qty and UC.

# Reset Sub-No row as there may be some unclean data by manually numbering rows.
systems['SN'] = np.nan
count = 1
for idx in systems.index:
    if pd.notna(systems.loc[idx, 'NO']):
        count = 1    # Reset count

    if pd.notna(systems.loc[idx, 'Description']) & pd.notna(systems.loc[idx, 'Qty']) & \
        pd.notna(systems.loc[idx, 'UC']) & (systems.loc[idx, 'Format'] != 'Item'):
        systems.at[idx, 'SN'] = count
        systems.at[idx, 'Format'] = 'Lineitem'
        count += 1
    if str(systems.loc[idx, 'Description']).startswith('***'):
        systems.at[idx, 'Format'] = 'Comment'

In [7]:
# Subtitle:
# TODO
for idx in systems.index:
    if idx == 0:
        continue
    if pd.notna(systems.loc[idx, 'Description']) and (systems.loc[idx+1, 'Format'] == 'Lineitem')\
        and pd.notna(systems.loc[idx+1, 'SN']) and pd.isnull(systems.loc[idx-1, 'Description'])\
        and pd.isnull(systems.loc[idx, 'SN']) and (systems.loc[idx, 'NO'] != 'Item'):
        systems.at[idx, 'Format'] = 'Subtitle'
    if idx == len(systems)-2:
        break

In [8]:
# Move Option and Included to scope
for idx in systems.index:
    if str(systems.loc[idx, 'Subtotal Price']).lower() in ['option', 'optional']:
        systems.at[idx, 'Scope'] = 'OPTION'
    if str(systems.loc[idx, 'Subtotal Price']).lower() in ['included','inclusive']:
        systems.at[idx, 'Scope'] = 'INCLUDED'
        

In [9]:
# Cleaning data
for idx in systems.index:
    if functions.set_nitty_gritty(str(systems.loc[idx, 'Description'])) != 'None':
        systems.at[idx, 'Description'] = functions.set_nitty_gritty(str(systems.loc[idx, 'Description']))
    if (str(systems.loc[idx, 'Model']).lower().strip() == 'start line:  delete forbidden'):
        systems.at[idx, 'Model'] = np.nan
    if (str(systems.loc[idx, 'UC']).lower().strip() == 'true' or str(systems.loc[idx, 'UC']).lower().strip() == 'false'):
        systems.at[idx, 'UC'] = np.nan   
    if (str(systems.loc[idx, 'SC']).lower().strip() == 'true' or str(systems.loc[idx, 'SC']).lower().strip() == 'false'):
        systems.at[idx, 'SC'] = np.nan

In [10]:
# Download template file from the internet and write to local folder
url = "https://filedn.com/liTeg81ShEXugARC7cg981h/Proposal_Template.xlsx"
resp = requests.get(url)

with open("Template.xlsx", 'wb') as fd:
    for chunk in resp.iter_content(chunk_size=8192):
        fd.write(chunk)

In [11]:
# Copy sheet from template to new workbook
new_book = xw.Book()
template = xw.Book("Template.xlsx", password=hide.new)
template.sheets['config'].copy(after=new_book.sheets[0])
new_book.sheets['Sheet1'].delete()
template.sheets['Cover'].copy(after=new_book.sheets['config'])

# Set date in Config
cell = new_book.sheets['config'].range('B32')
cell.value = datetime.today().strftime('%Y-%m-%d')

# Set up formula in Cover sheet
cell = new_book.sheets['Cover'].range('D7')
cell.formula = '=Config!B26'
cell = new_book.sheets['Cover'].range('C42')
cell.formula = '=Config!B21'
cell = new_book.sheets['Cover'].range('C43')
cell.formula = '=Config!B23'
cell = new_book.sheets['Cover'].range('C44')
cell.formula = '=Config!B24'
cell = new_book.sheets['Cover'].range('C45')
cell.formula = '=Config!B29'
cell = new_book.sheets['Cover'].range('C46')
cell.formula = '=Config!B30'
cell = new_book.sheets['Cover'].range('C47')
cell.value = '=Config!B32'

for system in system_names[::-1]:
    sheet_name = 'Cover'
    template.sheets['System'].copy(after=new_book.sheets[sheet_name])
    sheet_name = system
    new_book.sheets['System'].name = sheet_name
    # Set formula to reference Config.
    cell = new_book.sheets[sheet_name].range('C1')
    cell.formula = '=Config!B29'
    cell = new_book.sheets[sheet_name].range('C2')
    cell.formula = '=Config!B30'
    cell = new_book.sheets[sheet_name].range('C3')
    cell.formula = '=Config!B32'
    cell = new_book.sheets[sheet_name].range('C4')
    cell.formula = '=Config!B26'
# new_book.save( path + 'output.xlsx', password=hide.new)
# new_book.close()
template.close()

In [ ]:
template = xw.Book("Template.xlsx", password=hide.new)
template.sheets['System'].copy(after=new_book.sheets[system_names[-1]])
sheet_name = 'ES'
new_book.sheets['System'].name = sheet_name
# Set formula to reference Config.
cell = new_book.sheets[sheet_name].range('C1')
cell.formula = '=Config!B29'
cell = new_book.sheets[sheet_name].range('C2')
cell.formula = '=Config!B30'
cell = new_book.sheets[sheet_name].range('C3')
cell.formula = '=Config!B32'
cell = new_book.sheets[sheet_name].range('C4')
cell.formula = '=Config!B26'


In [12]:
for system in system_names:
    sheet = new_book.sheets[system]
    system = systems[systems['System'] == system]
    sheet.range('A5').options(index=False).value = system

In [13]:
# Set exchange rates
sheet = new_book.sheets['config']
exchange = pd.DataFrame([exchange_rates])
exchange = exchange.T
sheet.range('A2').value = exchange

# Quoted currency
sheet.range('B12').value = quoted_currency

In [14]:
# Write necessary formula to excel
for system in system_names:
    sheet = new_book.sheets[system]

    # Set default values
    sheet.range('I2').value = defaults[system]['default_mu']
    sheet.range('K1').value = defaults[system]['Default']
    sheet.range('K2').value = defaults[system]['Warranty']
    sheet.range('K3').value = defaults[system]['Inbound Freight']
    sheet.range('K4').value = defaults[system]['Special Terms']

    # Formula to cells
    last_row = sheet.range('C100000').end('up').row
    sheet.range('N6:N' + str(last_row)).formula = '=IF(K6<>"",K6*(1-M6),"")'
    sheet.range('O6:O' + str(last_row)).formula = '=IF(AND(D6<>"", K6<>""),D6*N6,"")'
    if quoted_currency == 'SGD' and exchange_rates['SGD'] == 1:
        sheet.range('Q6:Q' + str(last_row)).formula = '=IF(J6<>"",VLOOKUP(J6,Config!$A$2:$B$10,2,FALSE),"")'
    elif quoted_currency == 'USD' and exchange_rates['SGD'] == 1:
        sheet.range('Q6:Q' + str(last_row)).formula = '=IF(J6<>"",VLOOKUP(J6,Config!$A$2:$B$10, 2, FALSE) /VLOOKUP(Config!$B$12,Config!$A$2:$B$10,2,FALSE),"")'
    else:
        print('Currency does not seem right!')
    sheet.range('R6:R' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"") ,N6*Q6,"")'
    sheet.range('S6:S' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"") ,D6*R6,"")'
    sheet.range('T6:T' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""), (R6*(1+$K$1+$K$2+$K$3+$K$4))/(1-0.05),"")'
    # Below formula does not capture all options yet
    sheet.range('U6:U' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),IF(H6="OPTION","",D6*T6),"")'
    sheet.range('V6:V' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$1,"")'
    sheet.range('W6:W' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$2,"")'
    sheet.range('X6:X' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$3,"")'
    sheet.range('Y6:Y' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$4,"")'
    sheet.range('Z6:Z' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),T6-(R6+V6+W6+X6+Y6),"")'
    sheet.range('AA6:AA' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),$I$2,"")'
    sheet.range('AC6:AC' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),CEILING(T6/(1-AA6), 1),"")'
    sheet.range('AD6:AD' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),D6*AC6,"")'
    sheet.range('AE6:AE' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),IF(AB6<>"",AB6,AC6),"")'
    sheet.range('AF6:AF' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),D6*AE6,"")'
    sheet.range('AG6:AG' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"", H6<>"OPTION"),AF6-U6,"")'
    sheet.range('AH6:AH' + str(last_row)).formula = '=IF(AG6<>"",AG6/AF6,"")'
    sheet.range('F'+ str(last_row+2)).value = 'Subtotal (' + quoted_currency + ')'
    sheet.range('G' + str(last_row+2)).formula = '=SUM(G6:G' + str(last_row+1) + ')'
    sheet.range('U' + str(last_row+2)).formula = '=SUM(U6:U' + str(last_row+1) + ')'
    sheet.range('AF' + str(last_row+2)).formula = '=SUM(AF6:AF' + str(last_row+1) + ')'
    sheet.range('AG' + str(last_row+2)).formula = '=SUM(AG6:AG' + str(last_row+1) + ')'
    sheet.range('F6:F' + str(last_row)).formula = '=IF(AE6<>"", AE6,"")'
    sheet.range('G6:G' + str(last_row)).formula = '=IF(AF6<>"", AF6,"")'



